A pod is the lowest unit of an application in Kubernetes. Now, before we move on, we need to get one thing straight — and that is a pod is not equal to a container in the Docker world. A pod can be made up of multiple containers. If you have come from a pure Docker background, this can be hard to wrap your head around. If a pod can have more than one container, how does it work? There are some limits we need to be aware of. 

A pod has the following:

- A single IP address

- Share localhost

- A shared IPC space

- A shared network port range

- Shared volumes

The containers in a pod talk to each other via local host, whereas pod-to-pod communication is done via services.

deploy `nginx` as a Pod named `nginx-1`
```
kubectl create deployment --image nginx nginx-1
```
This command creates a Pod named nginx with a container running the nginx image. When a repository isn't specified, the default behavior is to try and find the image either locally or in the Docker public registry.

```
kubectl get pods
```

Pod phases:
- Pending, when image is pulling
- Running, starting, restaring or running continuously
- Succeeded, all containers have finished running successfully. In other words, they've terminated successfully and they won't be restarting.
- Failed
- Unkonwn, state of a pod simply cannot be retrieved. Probably because of a communication error between the master and the kubelet.
- CrashLoopBackOff, one of the containers in the pod exited unexpectedly, even after it was restarted at least once. Usually pod isn't configured correctly.

```
kubectl describe pod [POD_NAME]
```

Pod:  
Name  
Namespace   
Node name  
Labels  
Status  
IP Address, etc  
  
Container:  
State (waiting, running, terminated)  
Images  
Ports  
Commands  
Restart counts, etc  

```
kubectl exec [POD_NAME] -- [command]

kubectl exec demo env
kubectl exec demo ps aux
kubectl exec demo cat /proc/1/mounts
kubectl exec demo ls /

kubectl exec -it [POD_NAME] -- [command]

```

The -i argument tells kubectl to pass the terminal's standard input to the container, and the -t argument tells kubectl that the input is a TTY.

If you don't use these arguments, then the exact command will be executed in the remote container and returned immediately to your local shell.

If a pod has more one container, use the -c argument to specify the specific container to attach into the pod.

```
kubectl logs [POD_NAME]
```

logs inlcude

- stdout

    Standard output on the console

- stderr:

    Error messages

copy file to container in pod
```
kubectl cp ~/test.html $my_nginx_pod:/usr/share/nginx/html/test.html
```

This command copies the test.html file from the local home directory to the /usr/share/nginx/html directory of the first container in the nginx Pod. You could specify other containers in a multi-container Pod by using the -c option, followed by the name of the container.

Expose the Pod for testing
--

create a service to expose our nginx Pod externally
```
kubectl expose pod $my_nginx_pod --port 80 --type LoadBalancer
```

view the resources being used by the nginx Pod
```
kubectl top pods
```

deploy your manifest
```
kubectl apply -f ./new-nginx-pod.yaml
```

start an interactive bash shell in the nginx container
```
kubectl exec -it new-nginx /bin/bash
```

set up port forwarding from Cloud Shell to the nginx Pod (from port 10081 of the Cloud Shell VM to port 80 of the nginx container)
```
kubectl port-forward new-nginx 10081:80
```

```
curl http://127.0.0.1:10081/test.html
```

Affinity
--

Pod number one with label app:webserver has both a hard limit and a soft limit. In this pod, the hard limit has an anti-affinity rule, saying that other pods with the label app:webserver aren't allowed in the same zone.  
On the other hand, the soft limit has an affinity where it prefers to have other positive label app:cache on the same node.  
The cached pods however, prefer to be scheduled on the same node as a webserver but have a hard limit that prevents multiple cache pods from being deployed in the same zone.  
The resulting distribution has the two webserver pods repelling each other at the zone level and attracting cache pods at the node level.  
Both cache pods also repel each other at the zone level and attract webserver pods at the node level. 

Pod #1
(Label - app: webserver)

__Hard limit__ No app:webserver  
on the same zone

_Soft limit_: Prefer app:cache  
one the same node

Pod #2
(Label - app: webserver)

__Hard limit__ No app:webserver  
on the same zone

_Soft limit_: Prefer app:cache  
one the same node

Pod #3
(Label - app: cache)

__Soft limit__ Prefer app:webserver  
on the same node

_Hard limit_: No app:cache  
one the same zone

Pod #4
(Label - app: cache)

__Soft limit__ Prefer app:webserver  
on the same node

_Hard limit_: No app:cache  
one the same zone

---

Let's walk through the example. Pod one is already running on a node in zone one.  

Now pod two is being scheduled. Pods one and two repel each other based on the hard limit. Therefore, pod two is scheduled to run in a different zone, zone two and on a different node.  

Pod three has a soft limit preference to run on the same node as webserver pods, so it can be scheduled on the same node as either pod one or pod two. Here, it's scheduled to run alongside pod one.  

Finally, pod four has to be scheduled. It will be repelled from pod three with it's hard limit, but can run on a different zone alongside pod two. Additionally, the soft limit has a preference to run on the same node as a webserver pod instead of on a different node.  

This allows you to specify affinity rules to co-locate pods at different topological layers and control the distribution of pods across topological layers.

if the labels on nodes change, affinity and anti-affinity rules will NOT be applied to already-running Pods.